# Thermal Supercapacitor Simulation

This notebook demonstrates the thermal modeling capabilities of the simulation engine. We will simulate a supercapacitor under high current load and observe the temperature rise due to internal power dissipation.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Add project root to path to allow imports
project_root = Path("../../..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from simcore.dynamic.components.controlled_voltage_source import ControlledVoltageSource
from simcore.dynamic.components.supercapacitor import Supercapacitor
from simcore.dynamic.components.resistor import Resistor
from simcore.dynamic.network.graph import Node, NetworkGraph
from simcore.dynamic.network.network import Network
from simcore.dynamic.solver.integrate import run_sim

## Simulation Setup

We define the circuit parameters and create the network. The circuit consists of a voltage source connected to a supercapacitor.

In [ ]:
# Simulation parameters
dt = 0.01  # 10 ms
t_stop = 10.0 # 10 seconds

# Create nodes
vin = Node("vin")
gnd = Node("gnd", is_ground=True)

# Create graph
graph = NetworkGraph()
graph.add_branch("Vsrc", vin, gnd)
graph.add_branch("SC", vin, gnd)

# Create components
# High current to generate heat: 50V across the supercap
# Supercap parameters: high ESR for visible heating
sc = Supercapacitor(
    esr10ms=7e-3,
    capacitance=62,
    esr1s=10e-3,
    cells_per_module=1,
    modules_per_string=1,
    strings=1,
    tau_dl=0.03,
)
# Override thermal parameters for faster response in short sim
sc.C_th = 34000  # 
sc.R_th = 0.17  # forced air cooling

components = {
    "Vsrc": ControlledVoltageSource(V=162, R_internal=1e-3),
    "SC": sc
}

# Create network
net = Network(graph, components, dt=dt)

print(f"Nodes: {net.node_names}")
print(f"Branches: {net.branch_names}")
print(f"Initial State z0: {net.z0}")
print(f"A shape: {net.A.shape}")

## Run Simulation

We run the simulation. The engine will automatically solve for the consistent initial conditions (DC Operating Point) before starting the transient analysis.

In [ ]:
print("Running simulation...")
result = run_sim(net, t_stop=t_stop)
print("Simulation complete.")

## Results Analysis

We plot the node voltages, branch currents, power dissipation, and the thermal state (temperature).

In [ ]:
t = result.t

plt.figure(figsize=(10, 12))

# Plot Voltages
plt.subplot(4, 1, 1)
plt.plot(t, result.v_nodes[0], label="V_node")
plt.ylabel("Voltage [V]")
plt.legend()
plt.grid(True)

# Plot Current
plt.subplot(4, 1, 2)
plt.plot(t, result.i_branches[:, 0], label="I_branch")
plt.ylabel("Current [A]")
plt.legend()
plt.grid(True)

# Plot Power
plt.subplot(4, 1, 3)
try:
    plt.plot(t, result.power_dissipation, label="P_loss_total")
except AttributeError:
    # Fallback calculation
    plt.plot(t, result.i_branches[:, 0] * result.v_nodes[0], label="P_loss (V_node0 * I_branch0)")
plt.ylabel("Power [W]")
plt.legend()
plt.grid(True)

# Plot States (Temperature)
plt.subplot(4, 1, 4)
for i in range(result.z_hist.shape[1]):
    label = f"State {i}"
    # Try to guess which one is temperature (starts at 25.0)
    if abs(result.z_hist[0, i] - 25.0) < 1e-3:
        label += " (Temperature?)"
    plt.plot(t, result.z_hist[:, i], label=label)
    
plt.ylabel("State Value")
plt.xlabel("Time [s]")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()